# Load data

In [1]:
import pandas as pd
import string
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer

import plotly.express as px
import plotly.io as pio

In [2]:
df_test = pd.read_csv('test.csv')
df_test.head()

,lyrics,sentiment
0,eliminate incompetent difference dont exist ha...,negative
1,hey oh hey yeah hey baby hey baby phone ringin...,positive
2,shadow dream dream shadow analogy shadow dream...,positive
3,every day clouding mind every day world redefi...,positive
4,call upon innocent call upon weak call avatar ...,neutral


In [3]:
print(df_test.isnull().sum())
df_test = df_test.dropna()

lyrics       74
sentiment     0
dtype: int64


In [4]:
def unique(list1):
   # intilize a null list
     unique_list = []
   # traverse for all elements
     for x in list1:
         # check if exists in unique_list or not
         if x not in unique_list:
              unique_list.append(x)
     return unique_list

In [5]:
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

comp_score = []
sentiment = []

for i in df_test.loc[:,'lyrics']:
    sentiment.append(sia.polarity_scores(i))


df_test.loc[:,'sent_scores'] = sentiment
df_test.loc[:,'comp_score'] = df_test.loc[:,'sent_scores'].apply(lambda x: x['compound'])
df_test.loc[:,'sentiment_vader'] = df_test.loc[:,'comp_score'].apply(lambda x: 'positive' if x>0.2 else 'negative' if x<-0.2 else 'neutral')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/pereira/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [6]:
df = df_test.rename(columns={'comp_score': 'label_vader'})

# Data Visualization

In [7]:
df

,lyrics,sentiment,sent_scores,label_vader,sentiment_vader
0,eliminate incompetent difference dont exist ha...,negative,"{'neg': 0.238, 'neu': 0.735, 'pos': 0.028, 'co...",-0.9537,negative
1,hey oh hey yeah hey baby hey baby phone ringin...,positive,"{'neg': 0.064, 'neu': 0.477, 'pos': 0.459, 'co...",0.9994,positive
2,shadow dream dream shadow analogy shadow dream...,positive,"{'neg': 0.098, 'neu': 0.593, 'pos': 0.309, 'co...",0.9951,positive
3,every day clouding mind every day world redefi...,positive,"{'neg': 0.075, 'neu': 0.774, 'pos': 0.151, 'co...",0.9143,positive
4,call upon innocent call upon weak call avatar ...,neutral,"{'neg': 0.163, 'neu': 0.592, 'pos': 0.245, 'co...",0.9201,positive
...,...,...,...,...,...
47501,mama told didnt like rock roll said please mam...,positive,"{'neg': 0.036, 'neu': 0.801, 'pos': 0.163, 'co...",0.8820,positive
47502,feel life rut modern world much want remove pa...,neutral,"{'neg': 0.175, 'neu': 0.664, 'pos': 0.161, 'co...",-0.5856,negative
47503,many time said forever said love would always ...,neutral,"{'neg': 0.156, 'neu': 0.601, 'pos': 0.243, 'co...",0.9302,positive
47504,ive never happy see photograph graced company ...,negative,"{'neg': 0.157, 'neu': 0.523, 'pos': 0.32, 'com...",0.9325,positive


In [8]:
from sklearn.metrics import accuracy_score

def calculate_accuracy(df):
    # Verificar se as colunas "sentiment" e "sentiment_vader" existem no DataFrame
    if 'sentiment' not in df.columns or 'sentiment_vader' not in df.columns:
        print("As colunas 'sentiment' e 'sentiment_vader' são necessárias no DataFrame.")
        return None
    
    # Obter os valores das colunas de sentimento real e sentimento previsto
    true_sentiment = df['sentiment']
    predicted_sentiment = df['sentiment_vader']
    
    # Calcular a precisão (accuracy)
    accuracy = accuracy_score(true_sentiment, predicted_sentiment)
    
    return accuracy

In [9]:
accuracy_score = calculate_accuracy(df)
print("Accuracy of Vader: {:.2f}%".format(accuracy_score * 100))

Accuracy of Vader: 37.44%
